# Zestaw 10. Równania różniczkowe zwyczajne

In [72]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
from math import tan

## 1. Metoda Eulera i Rungego-Kutty

Zaimplementuj dwie metody rozwiazywania równań różniczkowych zwyczajnych:
- Eulera

In [176]:
def euler(f, u0, t0, x, step):
    h = (x-t0)/step
    u = u0
    
    t = t0
    
    for _ in range(step):
        u += h*f(t, u)
        t += h
    
    return u

- Rungego-Kutty rzędu 4

In [177]:
def runge_kutta(f, u0, t0, x, step):
    h = (x-t0)/step
    u = u0
    
    t = t0
    for i in range(step):
        k1 = h*f(t, u)
        k2 = h*f(t+h/2, u+k1/2)
        k3 = h*f(t+h/2, u+k2/2)
        k4 = h*f(t+h, u+k3)
        u += (k1 + 2*k2 + 2*k3 + k4)/6
        t += h
    
    return u

#### test

* x' = x

In [178]:
f = lambda t, x:t

In [179]:
euler(f, 0, 0, 1, 1000)

0.4995000000000005

In [180]:
runge_kutta(f, 0, 0, 1, 1000)

0.5000000000000002

In [181]:
scipy.integrate.solve_ivp(f, (0, 1), [0])['y'][0, -1]

0.5000000000000001

* x' = 

In [182]:
f = lambda t, x: 2 + (x-t-1)**2

In [183]:
u = lambda t: 1 + t + tan(t-1)

porównać z wynikiem analitycznem z Kincaida

In [184]:
euler(f, 2, 1, 1.5625, 1000)

3.1928062559481556

In [185]:
runge_kutta(f, 2, 1, 1.5625, 100)

3.1929376738362407

In [186]:
u(1.5625)

3.1929376738358846

In [187]:
scipy.integrate.solve_ivp(f, (1, 1.5625), [2])['y'][0, -1]

3.1929385148183353

## 2. Wahadło matematyczne

## 3. Wahadło podwójne

## Wnioski

- 
- 